In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import preprocessing

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 60)
pd.set_option('mode.chained_assignment',  None)

plt.rc('font',family='malgun gothic')

smart_card_data = pd.read_csv('./data/smart_card_data.csv')
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
bus_sttn = pd.read_csv('./data/bus_sttn.csv')
route_curve = pd.read_csv('./data/route_curve.csv', encoding = 'cp949')

### getrouteinfoall
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']] 

### bus_sttn
bus_sttn_df = bus_sttn[['bus_sttn_id', 'posx', 'posy']]

In [2]:
from hannam import data_preprocessing

smart_card_data = data_preprocessing.smart_card_preprocessing(smart_card_data)
smart_card_data.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,DAY,GETON_HOUR,GETOFF_HOUR
1,540104832,675,0,30300083,130010001,130759426,1,8001279,8001538,1250,0,2022-04-01 09:25:42,2022-04-01 09:37:29,1,9,9
3,741520439,676,0,30300043,130010015,130759609,1,8001428,8002717,1250,0,2022-04-01 07:32:15,2022-04-01 07:52:54,1,7,7
4,759623940,676,0,30300071,130010009,130759147,1,8001990,8001464,1250,0,2022-04-01 19:10:41,2022-04-01 19:15:39,1,19,19


In [91]:
def filter_day(exdata, date):
    exdata = exdata[exdata['DAY']==date]
    exdata.drop(labels = ['DAY'], axis = 1, inplace = True)
    return exdata

In [92]:
df_0428 = filter_day(smart_card_data,28)

In [119]:
getstationbyrouteall_df = getstationbyrouteall[['ROUTE_CD','BUSSTOP_NM','BUSSTOP_SEQ','TOTAL_DIST','BUS_NODE_ID','GPS_LATI','GPS_LONG']]
df_0428_merge = df_0428.merge(getrouteinfoall_df, left_on = 'BUS_ROUTE_ID', right_on = 'ROUTE_CD')
df_0428_102 = df_0428_merge[df_0428_merge['ROUTE_NO']=='102']
df_0428_102.drop(labels = ['TR_MEANS_TYPE', 'BUS_ROUTE_ID','ROUTE_NO','TRANSF_CNT', 'USR_TYPE','GETOFF_HOUR','ROUTE_CD'], axis = 1, inplace = True)
df_0428_102 = df_0428_102[['CARD_NO','BUS_CAR_ID','GETON_BUS_STTN_ID','GETON_DATETIME','GETOFF_BUS_STTN_ID','GETOFF_DATETIME','GETON_HOUR']]
df_0428_102 = df_0428_102.merge(getstationbyrouteall_df, left_on = 'GETON_BUS_STTN_ID', right_on = 'BUS_NODE_ID')
df_0428_102 = df_0428_102[['CARD_NO','BUS_CAR_ID','GETON_BUS_STTN_ID','BUSSTOP_NM','BUSSTOP_SEQ','TOTAL_DIST','GPS_LATI','GPS_LONG','GETON_DATETIME','GETON_HOUR','GETOFF_BUS_STTN_ID','GETOFF_DATETIME']]
df_0428_102.head(3)

,CARD_NO,BUS_CAR_ID,GETON_BUS_STTN_ID,BUSSTOP_NM,BUSSTOP_SEQ,TOTAL_DIST,GPS_LATI,GPS_LONG,GETON_DATETIME,GETON_HOUR,GETOFF_BUS_STTN_ID,GETOFF_DATETIME
0,540004451,130759012,8002339,우송대학교서캠퍼스,40,18388,36.337547,127.44774,2022-04-28 10:25:24,10,8002488,2022-04-28 10:28:24
1,540004451,130759012,8002339,우송대학교서캠퍼스,23,11341,36.337547,127.44774,2022-04-28 10:25:24,10,8002488,2022-04-28 10:28:24
2,316569309,130752011,8002339,우송대학교서캠퍼스,40,18388,36.337547,127.44774,2022-04-28 16:22:24,16,8003054,2022-04-28 16:31:53


In [109]:
df_0428_102_5 = df_0428_102[df_0428_102['GETON_HOUR']==5]
df_0428_102_5.head(3)

,CARD_NO,BUS_CAR_ID,GETON_BUS_STTN_ID,BUSSTOP_NM,BUSSTOP_SEQ,TOTAL_DIST,GPS_LATI,GPS_LONG,GETON_DATETIME,GETON_HOUR,GETOFF_BUS_STTN_ID,GETOFF_DATETIME
1297,741841361,130752035,8001443,대주파크빌,57,24948,36.342636,127.44803,2022-04-28 05:52:01,5,8005921,2022-04-28 06:22:51
1298,741841361,130752035,8001443,대주파크빌,45,20473,36.342636,127.44803,2022-04-28 05:52:01,5,8005921,2022-04-28 06:22:51
1359,776210888,130752005,8001443,대주파크빌,57,24948,36.342636,127.44803,2022-04-28 05:59:09,5,8001534,2022-04-28 06:02:29


In [110]:
df_0428_102_5['BUS_CAR_ID'].value_counts()

BUS_CAR_ID
130752045    396
130752035    159
130752002    135
130759024     80
130752074     53
130752053     38
130752011     22
130752005     14
130759011      4
Name: count, dtype: int64

In [118]:
getstationbyrouteall

,BUSSTOP_ENG_NM,BUSSTOP_NM,BUSSTOP_SEQ,BUSSTOP_TP,BUS_NODE_ID,BUS_STOP_ID,GPS_LATI,GPS_LONG,ROAD_NM,ROAD_NM_ADDR,ROUTE_CD,TOTAL_DIST
0,Chungnam National University,충대농대종점,1,1,8002736,42740,36.365704,127.352036,,,30300001,51
1,Hanbit Apts.,한빛아파트,2,,8002941,42720,36.362860,127.353004,,,30300001,434
2,"Chungnam National University, College of Agric...",충대농대입구,3,,8002735,42710,36.360420,127.352530,,,30300001,815
3,Gung-dong,궁동,4,,8001193,42700,36.360806,127.349540,대학로,,30300001,1090
4,Chungnam National University,충남대학교,5,,8002721,42600,36.361560,127.344120,,,30300001,1667
...,...,...,...,...,...,...,...,...,...,...,...,...
11626,---,세종시청.교육청.시의회,21,,8007295,51112,36.478638,127.288770,,,93000318,54535
11627,---,보람동.대평동,22,,8007297,51110,36.475502,127.281425,,,93000318,55236
11628,---,대평동,23,,8007400,76070,36.472170,127.276150,,,93000318,56007
11629,---,세종터미널,24,,8007058,51011,36.470710,127.273190,,,93000318,56604


In [117]:
BUS_102 = getstationbyrouteall_df[getstationbyrouteall_df['']]

,BUSSTOP_NM,BUSSTOP_SEQ,TOTAL_DIST,BUS_NODE_ID,GPS_LATI,GPS_LONG
0,충대농대종점,1,51,8002736,36.365704,127.352036
1,한빛아파트,2,434,8002941,36.362860,127.353004
2,충대농대입구,3,815,8002735,36.360420,127.352530
3,궁동,4,1090,8001193,36.360806,127.349540
4,충남대학교,5,1667,8002721,36.361560,127.344120
...,...,...,...,...,...,...
11626,세종시청.교육청.시의회,21,54535,8007295,36.478638,127.288770
11627,보람동.대평동,22,55236,8007297,36.475502,127.281425
11628,대평동,23,56007,8007400,36.472170,127.276150
11629,세종터미널,24,56604,8007058,36.470710,127.273190


In [116]:
df_0428_102_5[df_0428_102_5['BUS_CAR_ID']==130759024]

,CARD_NO,BUS_CAR_ID,GETON_BUS_STTN_ID,BUSSTOP_NM,BUSSTOP_SEQ,TOTAL_DIST,GPS_LATI,GPS_LONG,GETON_DATETIME,GETON_HOUR,GETOFF_BUS_STTN_ID,GETOFF_DATETIME
24389,574413507,130759024,8002720,충남대학교,16,6385,36.361360,127.34468,2022-04-28 05:57:54,5,8003006,2022-04-28 06:12:42
24390,574413507,130759024,8002720,충남대학교,10,3928,36.361360,127.34468,2022-04-28 05:57:54,5,8003006,2022-04-28 06:12:42
24391,574413507,130759024,8002720,충남대학교,56,25164,36.361360,127.34468,2022-04-28 05:57:54,5,8003006,2022-04-28 06:12:42
24392,574413507,130759024,8002720,충남대학교,50,22976,36.361360,127.34468,2022-04-28 05:57:54,5,8003006,2022-04-28 06:12:42
24393,574413507,130759024,8002720,충남대학교,85,54847,36.361360,127.34468,2022-04-28 05:57:54,5,8003006,2022-04-28 06:12:42
...,...,...,...,...,...,...,...,...,...,...,...,...
111372,758209550,130759024,8002589,주유소,51,29200,36.359062,127.30973,2022-04-28 05:46:03,5,8006001,2022-04-28 06:01:25
111373,758209550,130759024,8002589,주유소,47,21125,36.359062,127.30973,2022-04-28 05:46:03,5,8006001,2022-04-28 06:01:25
111374,758209550,130759024,8002589,주유소,77,50650,36.359062,127.30973,2022-04-28 05:46:03,5,8006001,2022-04-28 06:01:25
111375,758209550,130759024,8002589,주유소,22,16994,36.359062,127.30973,2022-04-28 05:46:03,5,8006001,2022-04-28 06:01:25


In [114]:
df_0428_102_5[df_0428_102_5['BUS_CAR_ID']==130752011].drop_duplicates('GETON_BUS_STTN_ID',keep='first').sort_values('GETON_DATETIME')

,CARD_NO,BUS_CAR_ID,GETON_BUS_STTN_ID,BUSSTOP_NM,BUSSTOP_SEQ,TOTAL_DIST,GPS_LATI,GPS_LONG,GETON_DATETIME,GETON_HOUR,GETOFF_BUS_STTN_ID,GETOFF_DATETIME
47074,739863143,130752011,8005881,대동역2번출구,52,23190,36.328762,127.44144,2022-04-28 05:58:21,5,8002100,2022-04-28 06:22:37
